In [9]:
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
from rasterio.mask import mask
from scipy.stats import entropy
from scipy import ndimage
from rasterio import features
import os
import json
from indexes_func import *

def calc_edge_diversity(polygons, src):
    """
    Calculates diversity indices based on landcover classes that intersect with the edges of each polygon.

    Parameters:
    polygons (geopandas.GeoDataFrame): GeoDataFrame containing the polygons.
    src (rasterio.DatasetReader): Rasterio dataset reader object for the landcover raster.

    Returns:
    pandas.DataFrame: DataFrame containing the calculated diversity indices for each polygon.
    """

    # Get the affine transformation matrix for the raster
    transform = src.transform

    # Initialize arrays to store the edge pixels and their corresponding landcover values
    edge_pixels = []
    edge_values = []

    # Loop through each polygon in the GeoDataFrame
    for idx, row in polygons.iterrows():
        # Get the geometry of the polygon
        geom = row.geometry

        # Convert the geometry to a list of shapely LineStrings representing the edges of the polygon
        edges = list(geom.boundary)

        # Loop through each edge of the polygon
        for edge in edges:
            # Convert the edge to a list of pixel coordinates
            pixels = list(map(lambda x: (~transform * x.coords[:][0], ~transform * x.coords[:][1]), edge))

            # Get the landcover values for the pixels along the edge
            values = list(features.rasterize(pixels, out_shape=src.shape, fill=0, transform=transform, all_touched=True, dtype=np.uint8, default_value=None, invert=False))
            edge_pixels += pixels
            edge_values += values

    # Convert the edge pixels and values to numpy arrays
    edge_pixels = np.array(edge_pixels)
    edge_values = np.array(edge_values)

    # Remove any edge pixels that fall outside the bounds of the raster
    in_bounds = (edge_pixels[:,0] >= 0) & (edge_pixels[:,0] < src.width) & (edge_pixels[:,1] >= 0) & (edge_pixels[:,1] < src.height)
    edge_pixels = edge_pixels[in_bounds]
    edge_values = edge_values[in_bounds]

    # Get the unique landcover values and their counts along the edges
    unique, counts = np.unique(edge_values, return_counts=True)

    # Calculate the total number of landcover cells along the edges
    total_cells = np.sum(counts)

    # Calculate the Shannon diversity index
    shannon_diversity = -np.sum([(count/total_cells) * np.log2(count/total_cells) for count in counts if count > 0])

    # Calculate the Simpson diversity index
    simpson_diversity = 1 - np.sum([(count/total_cells)**2 for count in counts])

    # Calculate the diversity index
    diversity_index = np.sum(np.square(counts/total_cells))

    # Calculate the number of landcover classes
    class_numb = unique.size

    # Calculate the Shannon evenness index
    if (np.log2(class_numb)>1):
        shannon_evenness = shannon_diversity / np.log2(class_numb)
    else : 
        shannon_evenness = '1 seule classe'

    # Calculate the dominance index
    dominance_index = np.max(counts) / total_cells

    # Create a DataFrame to store the results
    results = gpd.GeoDataFrame(polygons.geometry)

    # Add columns for the diversity indices


In [6]:
# Ignore "division by zeros" numpy errors 
# np.seterr(all='ignore')

# Define directories
### Get the path to the parent dir
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
#### Data dir
# build the path to the code dir
code_dir = os.path.join(parent_dir, "code")
#### Data dir
# build the path to the data dir
data_dir = os.path.join(parent_dir, "data")
#### Output dir
# build the path to the output dir
output_dir = os.path.join(parent_dir, "output")
# Create the dir if it doesn't exist
if not os.path.exists(output_dir):
    os.mkdir(output_dir)


params_f = os.path.join(code_dir, 'param.json')
with open(params_f, 'r') as f:
    params = json.load(f)

    if params['save_all_fields'].lower() == 'true':
        save_all_fields = True
    else:
        save_all_fields = False
    raster_name = params["raster_file_name"]
    vector_name = params["rpg_complete_name"]
    output_name = params["name_for_output_parcels"]

In [13]:
# Load the raster data and vector data
with rasterio.open(os.path.join(data_dir, raster_name)) as src:
    #Get infos about projection (in case data don't use the same projection the code will reproject vectorial data so that it matches raster projection)
    raster = src.read(1)
    r_crs = src.crs
    r_epsg = r_crs.to_epsg()
    print('Your raster file is using crs: ', r_epsg, 'This coordinate system will be used to reproject vectorial data if not.')
    
    
    polygons = gpd.read_file(os.path.join(data_dir, vector_name))
    print(polygons.crs)
    calc_edge_diversity(polygons, src)

Your raster file is using crs:  2154 This coordinate system will be used to reproject vectorial data if not.
PROJCS["RGF93 Lambert 93",GEOGCS["RGF93 geographiques (dms)",DATUM["Reseau_Geodesique_Francais_1993_v1",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6171"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["IGNF","RGF93G"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",46.5],PARAMETER["central_meridian",3],PARAMETER["standard_parallel_1",44],PARAMETER["standard_parallel_2",49],PARAMETER["false_easting",700000],PARAMETER["false_northing",6600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["IGNF","LAMB93"]]


TypeError: 'LineString' object is not iterable

In [11]:
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
from rasterio.mask import mask

# Load the raster data and vector data
with rasterio.open(os.path.join(data_dir, raster_name)) as src:
    raster = src.read(1)

polygons = gpd.read_file(os.path.join(data_dir, vector_name))

# Create a new column to store the edge-diversity index
polygons['edge_diversity_index'] = np.nan

# Loop through each polygon
for index, polygon in polygons.iterrows():
    # Extract the edges of the polygon as line segments
    edges = [LineString([pt1, pt2]) for pt1, pt2 in zip(polygon.geometry.exterior.coords[:-1], polygon.geometry.exterior.coords[1:])]
    print('1')
    
    # Clip the raster to the line segments
    edge_rasters = []
    for edge in edges:
        
        clipped_raster, _ = mask(rasterio.open(os.path.join(data_dir, raster_name)), [edge], crop=True)
        edge_rasters.append(clipped_raster)
    print('2')
    # Calculate the diversity index for each edge raster
    edge_diversity_indices = []
    for edge_raster in edge_rasters:
        unique_values = np.unique(edge_raster)
        total_cells = np.size(edge_raster)
        diversity_index = len(unique_values) / total_cells
        edge_diversity_indices.append(diversity_index)
    print('3')
    # Calculate the average edge-diversity index
    edge_diversity_index = np.mean(edge_diversity_indices)
    
    # Store the edge-diversity index in the new column
    polygons.loc[index, 'edge_diversity_index'] = edge_diversity_index

ValueError: Input shapes do not overlap raster.